In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import sys

In [2]:
PATH = "../data/data/train_fish/"
# path devided into kinds of image
ALB = "ALB"
BET = "BET"
DOL = "DOL"
LAG = "LAG"
NoF = "NoF"
OTHER = "OTHER"
SHARK = "SHARK"
YFT = "YFT"
FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
channnels=3

## データ

データはディレクトリごとに別れていて、合計で3777枚ある。

In [3]:
import os
for path, di, file  in os.walk(PATH):
    CLASS = di
    break
NUM_CLASSES = len(CLASS)
# IMAGE_SIZE = 256
IMAGE_SIZE = 32
IMAGE_PIXELS = IMAGE_SIZE*IMAGE_SIZE*3


learning_rate = 1e-4
max_steps = 120
BATCH_SIZE = 100

In [4]:
# これらコードは、http://kivantium.hateblo.jp/entry/2015/11/18/233834から得られる
def inference(images_placeholder,keep_prob):
    def weight_variable(shape):
        initial = tf.truncated_normal(shape,stddev=0.1)
        return tf.Variable(initial)
    def bias_variable(shape):
        initial = tf.constant(0.1,shape=shape)
        return tf.Variable(initial)
    def conv2d(x,W):
        return tf.nn.conv2d(x,W,[1,1,1,1],padding='SAME')
    def max_pool_2x2(x):
        return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#     x_image = tf.reshape(images_placeholder,[-1,256,256,3])
    x_image = tf.reshape(images_placeholder,[-1,IMAGE_SIZE,IMAGE_SIZE,3])
    #  32 * 32 * 3 tensorへ変換する

    with tf.name_scope('conv1') as scope:
        W_conv1 = weight_variable([5,5,3,32])
        b_conv1 = bias_variable([32])
        h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)

    with tf.name_scope('pool1') as scope:
        h_pool1 = max_pool_2x2(h_conv1)
        
    with tf.name_scope('conv2') as scope:
        W_conv2 = weight_variable([5,5,32,64])
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
        
    with tf.name_scope('pool2') as scope:
        h_pool2 = max_pool_2x2(h_conv2)
        # h_pool2 has (?, 64, 64, 64) shape
    #全結合
    with tf.name_scope('zenketsu1') as scope:
#         W_fc1 = weight_variable([64*64*64,1024])
        W_fc1 = weight_variable([64*64*1,1024])
        b_fc1 = bias_variable([1024])
#         h_pool2_flat = tf.reshape(h_pool2,[-1,64*64*64]) #3136 defined here
        h_pool2_flat = tf.reshape(h_pool2,[-1,64*64*1]) #3136 defined here
        
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)
        #dropout
        h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)
        
    with tf.name_scope('zenketsu2') as scope:
        W_fc2 = weight_variable([1024,NUM_CLASSES])
        b_fc2 = bias_variable([NUM_CLASSES])
    
    with tf.name_scope('softmax') as scope:
        y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)
    
    return y_conv

In [5]:
def loss(logits,labels):
    cross_entropy = -tf.reduce_sum(labels*tf.log(logits))
#     tf.scalar_summary("cross_entropy",cross_entropy)
    return cross_entropy

In [6]:
def training(loss,learning_rate):
    train_step = tf.train.AdadeltaOptimizer(learning_rate).minimize(loss)
    return train_step

In [7]:
# file load
def load_files(path):
    data = []
    save_pa = []
    for pa,_,_ in os.walk(path):
        if pa =='../data/data/train_fish/':
            continue
        save_pa.append(pa)
    for i_c, cat in enumerate(save_pa):
        for _,_,files in os.walk(cat):
            for file in files:
                label = np.zeros(8)
                label[i_c] = 1
                filepath = os.path.join(cat,file)
                data.append([filepath,label])
    return data
DATA = load_files(PATH)
import csv
with open("../data/meta.csv",'w') as f:
    writer = csv.writer(f,lineterminator='\n')
    writer.writerow(DATA)

使用しているデータの頻度を掛けないといけないかも。それ以外の解放だと、入れたデータについて、albが出る確率が0.5以上であるかどうか。

In [8]:
DATA[0]

['../data/data/train_fish/NoF/img_04395.jpg',
 array([1., 0., 0., 0., 0., 0., 0., 0.])]

In [44]:
DATA[0:10]

[['../data/data/train_fish/YFT/img_01472.jpg',
  array([0., 0., 0., 0., 0., 0., 1., 0.])],
 ['../data/data/train_fish/LAG/img_06223.jpg',
  array([0., 0., 0., 0., 0., 1., 0., 0.])],
 ['../data/data/train_fish/BET/img_07519.jpg',
  array([0., 0., 0., 0., 1., 0., 0., 0.])],
 ['../data/data/train_fish/BET/img_05344.jpg',
  array([0., 0., 0., 0., 1., 0., 0., 0.])],
 ['../data/data/train_fish/BET/img_02168.jpg',
  array([0., 0., 0., 0., 1., 0., 0., 0.])],
 ['../data/data/train_fish/ALB/img_01520.jpg',
  array([0., 0., 1., 0., 0., 0., 0., 0.])],
 ['../data/data/train_fish/ALB/img_07868.jpg',
  array([0., 0., 1., 0., 0., 0., 0., 0.])],
 ['../data/data/train_fish/ALB/img_06185.jpg',
  array([0., 0., 1., 0., 0., 0., 0., 0.])],
 ['../data/data/train_fish/ALB/img_02542.jpg',
  array([0., 0., 1., 0., 0., 0., 0., 0.])],
 ['../data/data/train_fish/NoF/img_02323.jpg',
  array([1., 0., 0., 0., 0., 0., 0., 0.])]]

In [9]:
len(DATA)

3777

精度が0.11で固定されていたので、学習量不足と判断し、データ数を増やした。

In [42]:
def take(data,num):
    reserved = []
    for v in data:
        if np.argmax(v[1]) == num:
            reserved.append(v)
    return reserved

In [11]:
def accuracy(logits,labels):
    correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
    return accuracy
def split_image_and_label(data_list,LESS=False):
    '''
    data_list should be [[data,label]...]
    '''
    data = []
    labels = []
    if LESS:
        i=0
        for value,label in data_list:
            i+=1
            data.append(value)
            labels.append(label)
            if i == 1000:
                break
        return data,labels
    else:
        for value,label in data_list:
            data.append(value)
            labels.append(label)
        return data, labels

In [12]:
# opnecvによる読み込み
import cv2 
def convert_image_path_to_data_cv2(path,i):
    try:
#         img2 = cv2.imread(path,0) # 白黒画像で読み込み
        img2 = cv2.imread(path)

        img2 = cv2.resize(img2,dsize=(IMAGE_SIZE,IMAGE_SIZE),interpolation = cv2.INTER_CUBIC)
        np_image_data = np.asarray(img2)
        np_final = np.expand_dims(np_image_data,axis=0)
        np_final = flatten(np_final)
        if i % 1000 == 0:
            print(i)
        #1000 images / 15s
        return np_final
    except:
        print(path)
        print(i)
        return np.zeros([256,256])

In [13]:
def flatten(image):
    image = image.flatten().astype(np.float32)/255.0
    return image

In [45]:
import random
random.shuffle(DATA)
num_train = int(len(DATA) * 0.8)
train = DATA[:num_train]
test = DATA[num_train:]

test = take(test,2)

In [46]:
test[0:10]

[['../data/data/train_fish/ALB/img_04755.jpg',
  array([0., 0., 1., 0., 0., 0., 0., 0.])],
 ['../data/data/train_fish/ALB/img_00269.jpg',
  array([0., 0., 1., 0., 0., 0., 0., 0.])],
 ['../data/data/train_fish/ALB/img_07576.jpg',
  array([0., 0., 1., 0., 0., 0., 0., 0.])],
 ['../data/data/train_fish/ALB/img_00526.jpg',
  array([0., 0., 1., 0., 0., 0., 0., 0.])],
 ['../data/data/train_fish/ALB/img_07863.jpg',
  array([0., 0., 1., 0., 0., 0., 0., 0.])],
 ['../data/data/train_fish/ALB/img_07811.jpg',
  array([0., 0., 1., 0., 0., 0., 0., 0.])],
 ['../data/data/train_fish/ALB/img_05443.jpg',
  array([0., 0., 1., 0., 0., 0., 0., 0.])],
 ['../data/data/train_fish/ALB/img_07459.jpg',
  array([0., 0., 1., 0., 0., 0., 0., 0.])],
 ['../data/data/train_fish/ALB/img_00975.jpg',
  array([0., 0., 1., 0., 0., 0., 0., 0.])],
 ['../data/data/train_fish/ALB/img_03524.jpg',
  array([0., 0., 1., 0., 0., 0., 0., 0.])]]

In [47]:
test_image,test_label = split_image_and_label(test,False) 
test_image = [convert_image_path_to_data_cv2(path,i) for i,path in enumerate(test_image)]

0


In [63]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('../data/model/model.ckpt.meta')
    saved_path = '../data/model/model.ckpt'
    ckpt = tf.train.get_checkpoint_state('../data/model/')
    last_model  = ckpt.model_checkpoint_path
    print("restoring")
    saver.restore(sess,last_model)
    print("model restored.")
#     print("you'll use these variables")
#     names = [tensor.name for tensor in tf.get_default_graph().as_graph_def().node]
#     print(names)
    graph = tf.get_default_graph()
#     prediction = graph.get_operation_by_name("inference")

#     images_placeholder = tf.placeholder("float",shape=(None,IMAGE_PIXELS))
#     labels_placeholder = tf.placeholder("float",shape=(None,NUM_CLASSES))
#     keep_prob = tf.placeholder("float")
    
    
    logits = inference(images_placeholder,keep_prob)
    loss_value = loss(logits,labels_placeholder)
    train_op = training(loss_value,learning_rate)
    acc = accuracy(logits,labels_placeholder)    
#test結果の答えと実際の値を見る
# test_imageは実際にデータを取っている。
    print("test accuracy %g"%sess.run(acc, feed_dict={
        images_placeholder: test_image,
        labels_placeholder: test_label,
        keep_prob: 1.0}))
    
    print(test_label[i])
    

restoring
INFO:tensorflow:Restoring parameters from ../data/model/model.ckpt
model restored.


FailedPreconditionError: Attempting to use uninitialized value conv2_423/Variable
	 [[node conv2_423/Variable/read (defined at <ipython-input-4-3ee9635687e4>:5)  = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](conv2_423/Variable)]]

Caused by op 'conv2_423/Variable/read', defined at:
  File "/home/genki/yes/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/genki/yes/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/genki/yes/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/genki/yes/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/genki/yes/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/genki/yes/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 132, in sta

In [ ]:
 for i,val in enumerate(test_image):
        result = inference(tmp,1.0)
        sess.run(result,feed_dict={images_placeholder:val})
        print(test_label[i])
        if i == 100:
            break

### 編集用